<a href="https://colab.research.google.com/github/jansle2005/Data-Science-Repository/blob/master/busca_pelo_twitter_sem_API_KEY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importação de Bibliotecas

In [ ]:
!pip install snscrape
!pip install pandas
!pip install nltk


## Importação das Bibliotecas

In [89]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta
import nltk
nltk.download('vader_lexicon')

## Definir os Parâmtros das Buscas no Twitter

In [113]:
# Definir parâmetros da busca
string_busca = "Ministério das Comunicações"
data_inic = "2022-01-20"
data_fim = "2022-02-18"
Limite = 200

# Converter data_inic e data_fim para objetos datetime
data_inic_dt = datetime.strptime(data_inic, '%Y-%m-%d')
data_fim_dt = datetime.strptime(data_fim, '%Y-%m-%d')

# Inicializar lista de tweets
tweets_list = []

# Criar intervalo de datas
datas = pd.date_range(data_inic, data_fim, freq='D')

## Fazer o Loop pelas Datas e fazer a busca

In [114]:
# Loop para buscar tweets
for data in datas:

    # Definir consulta para snscrape
    consulta = f'"{string_busca}" since:{data.date()} until:{(data + pd.Timedelta(days=1)).date()}'

    # Realizar a busca com snscrape
    for tweet in sntwitter.TwitterSearchScraper(consulta).get_items():
        # Adicionar os dados do tweet à lista
        tweets_list.append([tweet.date, tweet.content, tweet.user.username, tweet.user.location, tweet.user.followersCount])

        # Verificar se atingimos o limite de tweets desejado
        if len(tweets_list) >= Limite:
            break

    # Verificar se atingimos o limite de tweets desejado
    if len(tweets_list) >= Limite:
        break

# Converter a lista de tweets em um DataFrame
resultado = pd.DataFrame(tweets_list, columns=['Data', 'Tweet', 'Usuário', 'Localização', 'Seguidores'])
display(resultado)

,Data,Tweet,Usuário,Localização,Seguidores
0,2022-01-20 21:52:04+00:00,Parceria entre o Ministério das Comunicações ...,Aisha_com_vida,"Curitiba, Brasil",29967
1,2022-01-20 21:38:53+00:00,"@ernestofaraujo PR @jairbolsonaro, como o Gara...",wfaria92,"Santos, São Paulo - Brasil",701
2,2022-01-20 21:08:30+00:00,Parceria entre o Ministério das Comunicações ...,rj_decio,,3296
3,2022-01-20 20:23:45+00:00,Parceria entre o Ministério das Comunicações ...,OlivkovoyeCABR,"Vancouver, British Columbia",475
4,2022-01-20 19:12:54+00:00,😎👍🤝👉🇧🇷\nAção Gov. Bolsonaro\nParceria entre o ...,WaldemirodaSil4,"São Paulo, Brasil",1307
...,...,...,...,...,...
195,2022-02-13 13:00:21+00:00,O maior leilão de radiofrequências da América ...,assembleiase,"Sergipe, Brasil",4343
196,2022-02-13 11:37:55+00:00,fibra óptica. \n770 km de cabo de fibra óptica...,PapaiMauricio,"Santos, Brasil",1741
197,2022-02-14 13:23:29+00:00,Marco Feliciano apresentou projeto de lei que ...,aparecidoscarmo,"Cuiabá, Brasil",41
198,2022-02-14 10:36:22+00:00,Ministério das Comunicações comemora alta no s...,GlauciaLimaa,Caicó-RN,3732


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

## Fazer a Análise de Sentimento

In [116]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Instanciar o SentimentIntensityAnalyzer do NLTK
analyzer = SentimentIntensityAnalyzer()

# Adicionar coluna "sentimento" ao DataFrame "resultado"
resultado['sentimento_NLTK'] = resultado['Tweet'].apply(lambda tweet: analyzer.polarity_scores(tweet)['compound'])



In [136]:
from textblob import TextBlob

# função para calcular o número de palavras em cada tweet
def contar_palavras(tweet):
    return len(tweet.split())

# função para calcular a polaridade do tweet usando TextBlob
def calcular_polaridade(tweet):
    tb = TextBlob(tweet)
    return tb.sentiment.polarity

# função para calcular o sentimento do tweet usando TextBlob
def calcular_sentimento(tweet):
    tb = TextBlob(tweet)
    return tb.sentiment.subjectivity



# adicionar coluna com o número de palavras de cada tweet
resultado['num_palavras_Blob'] = resultado['Tweet'].apply(contar_palavras)

# adicionar coluna com a polaridade de cada tweet
resultado['polaridade_Blob'] = resultado['Tweet'].apply(calcular_polaridade)

# adicionar coluna com o sentimento de cada tweet
resultado['sentimento_Blob'] = resultado['Tweet'].apply(calcular_sentimento)


## Mostrar o Resultado

In [138]:
display(resultado)

,Data,Tweet,Usuário,Localização,Seguidores,sentimento,num_palavras,polaridade,sentimentoV
0,2022-01-20 21:52:04+00:00,Parceria entre o Ministério das Comunicações ...,Aisha_com_vida,"Curitiba, Brasil",29967,0.0,42,0.0,0.0000
1,2022-01-20 21:38:53+00:00,"@ernestofaraujo PR @jairbolsonaro, como o Gara...",wfaria92,"Santos, São Paulo - Brasil",701,0.0,14,0.0,-0.2960
2,2022-01-20 21:08:30+00:00,Parceria entre o Ministério das Comunicações ...,rj_decio,,3296,0.0,42,0.0,0.0000
3,2022-01-20 20:23:45+00:00,Parceria entre o Ministério das Comunicações ...,OlivkovoyeCABR,"Vancouver, British Columbia",475,0.0,28,0.0,0.0000
4,2022-01-20 19:12:54+00:00,😎👍🤝👉🇧🇷\nAção Gov. Bolsonaro\nParceria entre o ...,WaldemirodaSil4,"São Paulo, Brasil",1307,0.0,46,0.0,0.4588
...,...,...,...,...,...,...,...,...,...
195,2022-02-13 13:00:21+00:00,O maior leilão de radiofrequências da América ...,assembleiase,"Sergipe, Brasil",4343,0.0,36,0.0,0.0000
196,2022-02-13 11:37:55+00:00,fibra óptica. \n770 km de cabo de fibra óptica...,PapaiMauricio,"Santos, Brasil",1741,0.0,31,0.0,-0.2960
197,2022-02-14 13:23:29+00:00,Marco Feliciano apresentou projeto de lei que ...,aparecidoscarmo,"Cuiabá, Brasil",41,0.0,39,0.0,0.0000
198,2022-02-14 10:36:22+00:00,Ministério das Comunicações comemora alta no s...,GlauciaLimaa,Caicó-RN,3732,0.0,10,0.0,-0.2960
